In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = load_breast_cancer()

In [ ]:
X = data.data

In [ ]:
X

In [ ]:
y = data.target

In [ ]:
scaler = MinMaxScaler((0,1))
scaler.fit(X)
X_t = scaler.transform(X)

In [ ]:
xdf = pd.DataFrame(X_t)

In [ ]:
xdf.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y, test_size=0.2, random_state=42)

In [ ]:
sk_model = linear_model.LogisticRegression()

In [ ]:
%%time
sk_model.fit(X_train, y_train)

In [ ]:
%%time
preds = sk_model.predict(X_test)

In [ ]:
print("Accuracy: ",accuracy_score(y_test, preds))
print("F1 Score: ",f1_score(y_test, preds))
print("Precision: ",precision_score(y_test, preds))
print("Recall: ",recall_score(y_test, preds))

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.power(np.e,-z))

In [ ]:
def approxsigmoid(z):
    return 1 / (1 + np.power(2,-z))

In [ ]:
def TaylorSigmoid(z):
    return (0.5+0.25*z-np.power(z, 3)/48.+np.power(z,5)/480.)

In [ ]:
t = np.linspace(-2,2,1000)

In [ ]:
plt.plot(t, sigmoid(t),)
plt.title("Sigmoid")

In [ ]:
plt.plot(t, approxsigmoid(t))
plt.title("e=>2 Approximation")

In [ ]:
plt.plot(t, TaylorSigmoid(t))
plt.title("Taylor Approximation")

In [ ]:
%%timeit
res = sigmoid(t)

In [ ]:
%%timeit
ares = approxsigmoid(t)

In [ ]:
%%timeit
tres = TaylorSigmoid(t)

In [ ]:
res = sigmoid(t)
ares = approxsigmoid(t)
tres = TaylorSigmoid(t)

In [ ]:
print("Taylor Approximation Error: ", mean_absolute_error(tres,res))
print("e Approximation Error: ", mean_absolute_error(ares,res))

In [ ]:
class LogisticRegression:
    
    # defining parameters such as learning rate, number ot iterations, whether to include intercept, 
    # and verbose which says whether to print anything or not like, loss etc.
    def __init__(self, learning_rate=0.01, num_iterations=50000, fit_intercept=True, verbose=False):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    # function to define the Incercept value.
    def __b_intercept(self, X):
        # initially we set it as all 1's
        intercept = np.ones((X.shape[0], 1))
        # then we concatinate them to the value of X, we don't add we just append them at the end.
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid_function(self, z):
        # this is our actual sigmoid function which predicts our yp
        return 1 / (1 + np.power(np.e,-z))
    
    def __loss(self, yp, y):
        # this is the loss function which we use to minimize the error of our model
        return (-y * np.log(yp) - (1 - y) * np.log(1 - yp)).mean()
    
    # this is the function which trains our model.
    def fit(self, X, y):
        
        # as said if we want our intercept term to be added we use fit_intercept=True
        if self.fit_intercept:
            X = self.__b_intercept(X)
        
        # weights initialization of our Normal Vector, initially we set it to 0, then we learn it eventually
        self.W = np.zeros(X.shape[1])
        
        # this for loop runs for the number of iterations provided
        for i in range(self.num_iterations):
            
            # this is our W * Xi
            z = np.dot(X, self.W)
            
            # this is where we predict the values of Y based on W and Xi
            yp = self.__sigmoid_function(z)
            
            # this is where the gradient is calculated form the error generated by our model
            gradient = np.dot(X.T, (yp - y)) / y.size
            
            # this is where we update our values of W, so that we can use the new values for the next iteration
            self.W -= self.learning_rate * gradient
            
            # this is our new W * Xi
            z = np.dot(X, self.W)
            yp = self.__sigmoid_function(z)
            
            # this is where the loss is calculated
            loss = self.__loss(yp, y)
            
            # as mentioned above if we want to print somehting we use verbose, so if verbose=True then our loss get printed
            if(self.verbose ==True and i % 5000 == 0):
                print(f'loss: {loss} \t')
    
    # this is where we predict the probability values based on out generated W values out of all those iterations.
    def predict_prob(self, X):
        # as said if we want our intercept term to be added we use fit_intercept=True
        if self.fit_intercept:
            X = self.__b_intercept(X)
        
        # this is the final prediction that is generated based on the values learned.
        return self.__sigmoid_function(np.dot(X, self.W))
    
    # this is where we predict the actual values 0 or 1 using round. anything less than 0.5 = 0 or more than 0.5 is 1
    def predict(self, X):
        return self.predict_prob(X).round()

In [ ]:
scratch_model = LogisticRegression(verbose=True)

In [ ]:
%%time
scratch_model.fit(X_train,y_train)

In [ ]:
scratch_loss = [0.6802027676278086,0.1437848188409872,0.12231158389813686,0.11103282957702557,0.10352850820742168,
                0.09804229644421379,0.09380104663005029,0.09038101808681631,0.08753083896144674,0.08509315759390447 ]

In [ ]:
%%time
scratch_lr_preds = scratch_model.predict(X_test)

In [ ]:
print("Accuracy: ",accuracy_score(y_test, scratch_lr_preds))
print("F1 Score: ",f1_score(y_test, scratch_lr_preds))
print("Precision: ",precision_score(y_test, scratch_lr_preds))
print("Recall: ",recall_score(y_test, scratch_lr_preds))

In [ ]:
class ApproximateRegression:
    
    # defining parameters such as learning rate, number ot iterations, whether to include intercept, 
    # and verbose which says whether to print anything or not like, loss etc.
    def __init__(self, learning_rate=0.01, num_iterations=50000, fit_intercept=True, verbose=False):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    # function to define the Incercept value.
    def __b_intercept(self, X):
        # initially we set it as all 1's
        intercept = np.ones((X.shape[0], 1))
        # then we concatinate them to the value of X, we don't add we just append them at the end.
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid_function(self, z):
        # this is our actual sigmoid function which predicts our yp
        return 1 / (1 + np.power(2,-z))
    
    def __loss(self, yp, y):
        # this is the loss function which we use to minimize the error of our model
        return (-y * np.log(yp) - (1 - y) * np.log(1 - yp)).mean()
    
    # this is the function which trains our model.
    def fit(self, X, y):
        
        # as said if we want our intercept term to be added we use fit_intercept=True
        if self.fit_intercept:
            X = self.__b_intercept(X)
        
        # weights initialization of our Normal Vector, initially we set it to 0, then we learn it eventually
        self.W = np.zeros(X.shape[1])
        
        # this for loop runs for the number of iterations provided
        for i in range(self.num_iterations):
            
            # this is our W * Xi
            z = np.dot(X, self.W)
            
            # this is where we predict the values of Y based on W and Xi
            yp = self.__sigmoid_function(z)
            
            # this is where the gradient is calculated form the error generated by our model
            gradient = np.dot(X.T, (yp - y)) / y.size
            
            # this is where we update our values of W, so that we can use the new values for the next iteration
            self.W -= self.learning_rate * gradient
            
            # this is our new W * Xi
            z = np.dot(X, self.W)
            yp = self.__sigmoid_function(z)
            
            # this is where the loss is calculated
            loss = self.__loss(yp, y)
            
            # as mentioned above if we want to print somehting we use verbose, so if verbose=True then our loss get printed
            if(self.verbose ==True and i % 5000 == 0):
                print(f'loss: {loss} \t')

    
    # this is where we predict the probability values based on out generated W values out of all those iterations.
    def predict_prob(self, X):
        # as said if we want our intercept term to be added we use fit_intercept=True
        if self.fit_intercept:
            X = self.__b_intercept(X)
        
        # this is the final prediction that is generated based on the values learned.
        return self.__sigmoid_function(np.dot(X, self.W))
    
    # this is where we predict the actual values 0 or 1 using round. anything less than 0.5 = 0 or more than 0.5 is 1
    def predict(self, X):
        return self.predict_prob(X).round()

In [ ]:
approx_model = ApproximateRegression(verbose=True)

In [ ]:
%%time
approx_model.fit(X_train,y_train)

In [ ]:
approx_loss = [0.6838808388499469,0.15668311336660568,0.13327044900788637,0.12119362980100588,0.11316384088637117,
              0.10721375561196354,0.10254874031747165,0.09876261054734893,0.09560845558233506,0.09292325821026125]

In [ ]:
%%time
approx_lr_preds = approx_model.predict(X_test)

In [ ]:
print("Accuracy: ",accuracy_score(y_test, approx_lr_preds))
print("F1 Score: ",f1_score(y_test, approx_lr_preds))
print("Precision: ",precision_score(y_test, approx_lr_preds))
print("Recall: ",recall_score(y_test, approx_lr_preds))

In [ ]:
class TaylorRegression:
    
    # defining parameters such as learning rate, number ot iterations, whether to include intercept, 
    # and verbose which says whether to print anything or not like, loss etc.
    def __init__(self, learning_rate=0.01, num_iterations=50000, fit_intercept=True, verbose=False):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    # function to define the Incercept value.
    def __b_intercept(self, X):
        # initially we set it as all 1's
        intercept = np.ones((X.shape[0], 1))
        # then we concatinate them to the value of X, we don't add we just append them at the end.
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid_function(self, z):
        # this is our actual sigmoid function which predicts our yp
        res = (0.5+0.25*z-np.power(z,3)/48 + np.power(z,5)/480)
        return np.nan_to_num(res)
    
    def __loss(self, yp, y):
        # this is the loss function which we use to minimize the error of our model
        return np.nan_to_num((-y * np.log(yp) - (1 - y) * np.log(1 - yp))).mean()
    
    # this is the function which trains our model.
    def fit(self, X, y):
        
        # as said if we want our intercept term to be added we use fit_intercept=True
        if self.fit_intercept:
            X = self.__b_intercept(X)
        
        # weights initialization of our Normal Vector, initially we set it to 0, then we learn it eventually
        self.W = np.zeros(X.shape[1])
        
        # this for loop runs for the number of iterations provided
        for i in range(self.num_iterations):
            
            # this is our W * Xi
            z = np.dot(X, self.W)
            
            # this is where we predict the values of Y based on W and Xi
            yp = self.__sigmoid_function(z)
            
   
            
            # this is where the gradient is calculated form the error generated by our model
            gradient = np.dot(X.T, (yp - y)) / y.size
            
            # this is where we update our values of W, so that we can use the new values for the next iteration
            self.W -= self.learning_rate * gradient
            
            # this is our new W * Xi
            z = np.dot(X, self.W)
            yp = self.__sigmoid_function(z)
 
            
            # this is where the loss is calculated
            loss = self.__loss(yp, y)
            #print("loss: ", loss)
            
            # as mentioned above if we want to print somehting we use verbose, so if verbose=True then our loss get printed
            if(self.verbose ==True and i % 5000 == 0):
                print(f'loss: {loss} \t')
                

    
    # this is where we predict the probability values based on out generated W values out of all those iterations.
    def predict_prob(self, X):
        # as said if we want our intercept term to be added we use fit_intercept=True
        if self.fit_intercept:
            X = self.__b_intercept(X)
        
        # this is the final prediction that is generated based on the values learned.
        return self.__sigmoid_function(np.dot(X, self.W))
    
    # this is where we predict the actual values 0 or 1 using round. anything less than 0.5 = 0 or more than 0.5 is 1
    def predict(self, X):
        return self.predict_prob(X).round()

In [ ]:
taylor_model = TaylorRegression(verbose=True)

In [ ]:
%%time
taylor_model.fit(X_train,y_train)

In [ ]:
taylor_loss = [0.6802027675850325, 0.24888153461793144,0.23565447667633324, 0.22783043968681171,0.22211908628137508,
              0.21780747243903473, 0.21434131655968433, 0.2115019167958568, 0.2091635430385828, 0.2071597232944494]

In [ ]:
%%time
taylor_lr_preds = taylor_model.predict(X_test)

In [ ]:
taylor_lr_preds

In [ ]:
for i in range(len(taylor_lr_preds)):
    if taylor_lr_preds[i]<0:
        taylor_lr_preds[i] = 0
    elif taylor_lr_preds[i]>1:
        taylor_lr_preds[i] = 1

In [ ]:
taylor_lr_preds

In [ ]:
print("Accuracy: ",accuracy_score(y_test, taylor_lr_preds))
print("F1 Score: ",f1_score(y_test, taylor_lr_preds))
print("Precision: ",precision_score(y_test, taylor_lr_preds))
print("Recall: ",recall_score(y_test, taylor_lr_preds))

In [ ]:
plt.plot(scratch_loss, label="original")
plt.plot(approx_loss, label="e=>2 approx")
plt.plot(taylor_loss, label="taylor approx")
plt.legend()